In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)


In [ ]:
data = pd.read_excel("Dataset Comex.xlsx")
print("Shape:", data.shape)
data.head()


Shape: (29608, 17)


,Partner Name,Year,Export (US$ Thousand),Import (US$ Thousand),World Growth (%),Country Growth (%),latitude,longitude,CommercialMarket_Label,Median_time_in_port_days_Value,Average_size_GT_of_vessels_Value,Average_cargo_carrying_capacity_dwt_per_vessel_Value,Average_container_carrying_capacity_TEU_per_container_ship_Value,Maximum_size_GT_of_vessels_Value,Maximum_cargo_carrying_capacity_dwt_of_vessels_Value,Maximum_container_carrying_capacity_TEU_of_container_ships_Value,period
0,aruba,1988,3498.10,328.49,NaN,NaN,12.521110,-69.968338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,afghanistan,1988,213030.40,54459.52,NaN,NaN,33.939110,67.709953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,angola,1988,375527.89,370702.76,NaN,NaN,-11.202692,17.873887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,anguila,1988,366.98,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,albania,1988,30103.56,47709.30,NaN,NaN,41.153332,20.168331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

missing = data.isnull().sum().sort_values(ascending=False)
missing[missing > 0].head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29608 entries, 0 to 29607
Data columns (total 17 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Partner Name                                                      29608 non-null  object 
 1   Year                                                              29608 non-null  int64  
 2   Export (US$ Thousand)                                             29608 non-null  float64
 3   Import (US$ Thousand)                                             29608 non-null  float64
 4   World Growth (%)                                                  23619 non-null  float64
 5   Country Growth (%)                                                23619 non-null  float64
 6   latitude                                                          25458 non-null  float64
 7   longitude                      

,0
Maximum_container_carrying_capacity_TEU_of_container_ships_Value,24504
Average_container_carrying_capacity_TEU_per_container_ship_Value,24504
Average_cargo_carrying_capacity_dwt_per_vessel_Value,12562
Maximum_cargo_carrying_capacity_dwt_of_vessels_Value,12562
Median_time_in_port_days_Value,12442
period,7458
CommercialMarket_Label,7458
Average_size_GT_of_vessels_Value,7458
Maximum_size_GT_of_vessels_Value,7458
Country Growth (%),5989


In [ ]:
target = "Export (US$ Thousand)"

# Copia para modelado
df = data.copy()

# Eliminamos filas sin target (por seguridad)
df = df.dropna(subset=[target])

# y transformada (log1p)
y = np.log1p(df[target])

# X: todo menos target
X = df.drop(columns=[target])

print("X:", X.shape, "| y:", y.shape)


X: (29608, 16) | y: (29608,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (23686, 16) Test: (5922, 16)


In [ ]:
num_cols = X.select_dtypes(include=["number"]).columns
cat_cols = X.select_dtypes(exclude=["number"]).columns

print("Numéricas:", len(num_cols))
print("Categóricas:", len(cat_cols))
cat_cols


Numéricas: 13
Categóricas: 3


Index(['Partner Name', 'CommercialMarket_Label', 'period'], dtype='object')

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)


In [ ]:
# Entrenamiento
pipe.fit(X_train, y_train)

# Predicción
y_pred = pipe.predict(X_test)

y_pred[:5]



array([17.55221183, 17.5615671 , 17.67217005, 17.51589662, 17.51590333])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)   # MSE
rmse = np.sqrt(mse)                        # RMSE (compatible siempre)

r2 = r2_score(y_test, y_pred)

print(f"MAE  (log1p): {mae:.4f}")
print(f"RMSE (log1p): {rmse:.4f}")
print(f"R²          : {r2:.4f}")


MAE  (log1p): 1.8457
RMSE (log1p): 2.4726
R²          : 0.1851


In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(pipe, X, y, cv=5, scoring="r2")
print("R² CV (5 folds):", cv_scores)
print("Promedio R² CV:", cv_scores.mean().round(4))
print("Desv. Std R² CV:", cv_scores.std().round(4))


R² CV (5 folds): [-0.25189378  0.11738809  0.17708267  0.12797077  0.08721475]
Promedio R² CV: 0.0516
Desv. Std R² CV: 0.1545


In [ ]:
pre = pipe.named_steps["preprocess"]
feature_names = pre.get_feature_names_out()

mask = pipe.named_steps["select"].get_support()
selected = feature_names[mask]

print("Total features tras OHE:", len(feature_names))
print("Seleccionadas:", len(selected))
selected[:40]


## Conclusiones

- Se aplicó **feature selection** con `SelectKBest (f_regression)` para reducir la dimensionalidad a **K=30** variables (incluyendo variables numéricas y columnas generadas por one-hot encoding).  
- Se entrenó un modelo de **regresión Ridge**, adecuado cuando hay muchas variables y posible multicolinealidad.
- En el conjunto de test, el modelo obtuvo:
  - **MAE (log1p):** __REEMPLAZAR__
  - **RMSE (log1p):** __REEMPLAZAR__
  - **R²:** __REEMPLAZAR__
- La **validación cruzada (5 folds)** entregó un R² promedio de __REEMPLAZAR__ (std __REEMPLAZAR__), lo que sugiere un desempeño (estable / inestable) según dispersión.
- El gráfico *Real vs Predicho* permite ver el ajuste general. Si los puntos están cerca de la diagonal, el modelo captura bien la tendencia; desvíos en extremos pueden deberse a outliers o relaciones no lineales.

### Próximos pasos
- Probar otros algoritmos (RandomForestRegressor, GradientBoosting, etc.).
- Ajustar `K` (20/50/100) y comparar métricas.
- Hacer ingeniería de variables (por ejemplo ratio Export/Import, agregación por país, etc.).
